<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><span><a href="#Diversity-using-hill-numbers-from-immunarch" data-toc-modified-id="Diversity-using-hill-numbers-from-immunarch-4.1">Diversity using hill numbers from immunarch</a></span></li><li><span><a href="#Create-metadata-for-immunarch" data-toc-modified-id="Create-metadata-for-immunarch-4.2">Create metadata for immunarch</a></span></li></ul></li></ul></div>

# Dependencies

In [1]:
library(ggplot2)
library(immunarch)
library(ggbeeswarm)

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: dtplyr

Warning message:
“package ‘dtplyr’ was built under R version 4.0.5”
Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: patchwork



# Functions

In [2]:
histp <- function(df, var){
    myp <- ggplot(data = df, aes(x = eval(parse(text = var)))) + 
    geom_histogram() + myplot + myaxis + labs(x = var)
    return(myp)
}

In [3]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")

# Paths

In [4]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

# Main

In [5]:
# Copied back from h4h
load(paste0(datapath, "capTCRseq/int_load.RData"))

In [6]:
#remove scores from V and J
for(i in 1:length(int_load$data)){
    int_load$data[[i]] <- as.data.frame(int_load$data[[i]])
    int_load$data[[i]][,"V.name"] <- gsub("\\*00\\(.*", "", int_load$data[[i]][,"V.name"])
    int_load$data[[i]][,"D.name"] <- gsub("\\*00\\(.*", "", int_load$data[[i]][,"D.name"])
    int_load$data[[i]][,"J.name"] <- gsub("\\*00\\(.*", "", int_load$data[[i]][,"J.name"])
    int_load$data[[i]] <- as_tibble(int_load$data[[i]])
}

In [7]:
# names cleanup
names(int_load$data) <- gsub("CLONES_TRB", "", names(int_load$data))

In [8]:
int_load$meta$Sample <- gsub("CLONES_TRB", "", int_load$meta$Sample)

In [9]:
head(names(int_load$data))

[1] "CHP_10YO-M_PBMC_DNA"   "CHP_11YO-M_PBMC_DNA"   "CHP_15YO-F-A_PBMC_DNA"
[4] "CHP_15YO-F-B_PBMC_DNA" "CHP_15YO-M_PBMC_DNA"   "CHP_16YO-M_PBMC_DNA"

In [10]:
head(int_load$meta)

Sample
<chr>
CHP_10YO-M_PBMC_DNA
CHP_11YO-M_PBMC_DNA
CHP_15YO-F-A_PBMC_DNA
CHP_15YO-F-B_PBMC_DNA
CHP_15YO-M_PBMC_DNA
CHP_16YO-M_PBMC_DNA


## Diversity using hill numbers from immunarch

In [11]:
load(file = paste0(datapath,"capTCRseq/meta_div_TRB.RData"))

In [12]:
div_hill_aa <- repDiversity(int_load$data, "hill", .min.q = 0)
div_hill_v <- repDiversity(int_load$data, "hill", .col = "v", .min.q = 0)
div_hill_j <- repDiversity(int_load$data, "hill", .col = "j", .min.q = 0)

In [13]:
div_hill_aav <- repDiversity(int_load$data, "hill", .col = "aa+v", .min.q = 0)

Add to divstats

In [14]:
q0_aa <- as.data.frame(div_hill_aa[ div_hill_aa$Q == 0,])
q0_v <- as.data.frame(div_hill_v[ div_hill_v$Q == 0,])
q0_j <- as.data.frame(div_hill_j[ div_hill_j$Q == 0,])
q0_aav <- as.data.frame(div_hill_aav[ div_hill_aav$Q == 0,])

In [15]:
head(q0_aa,15)

,Sample,Q,Value
,<fct>,<dbl>,<dbl>
1,CHP_10YO-M_PBMC_DNA,0,22
2,CHP_11YO-M_PBMC_DNA,0,21
3,CHP_15YO-F-A_PBMC_DNA,0,13
4,CHP_15YO-F-B_PBMC_DNA,0,9
5,CHP_15YO-M_PBMC_DNA,0,33
6,CHP_16YO-M_PBMC_DNA,0,7
7,CHP_1YO-M-A_PBMC_DNA,0,65
8,CHP_1YO-M-B_PBMC_DNA,0,5
9,CHP_2YO-M_PBMC_DNA,0,16


In [16]:
meta_div$q0_aa <- q0_aa$Value[match(meta_div$basename, q0_aa$Sample)]
meta_div$q0_v <- q0_v$Value[match(meta_div$basename, q0_v$Sample)]
meta_div$q0_j <- q0_j$Value[match(meta_div$basename, q0_j$Sample)]
meta_div$q0_aav <- q0_aav$Value[match(meta_div$basename, q0_aav$Sample)]
meta_div$q0_vj <- rowMeans(meta_div[,c("q0_j", "q0_v")])

In [17]:
q1_aa <- as.data.frame(div_hill_aa[ div_hill_aa$Q == 1,])
q1_v <- as.data.frame(div_hill_v[ div_hill_v$Q == 1,])
q1_j <- as.data.frame(div_hill_j[ div_hill_j$Q == 1,])
q1_aav <- as.data.frame(div_hill_aav[ div_hill_aav$Q == 1,])

In [18]:
meta_div$q1_aa <- q1_aa$Value[match(meta_div$basename, q1_aa$Sample)]
meta_div$q1_v <- q1_v$Value[match(meta_div$basename, q1_v$Sample)]
meta_div$q1_j <- q1_j$Value[match(meta_div$basename, q1_j$Sample)]
meta_div$q1_aav <- q1_aav$Value[match(meta_div$basename, q1_aav$Sample)]
meta_div$q1_vj <- rowMeans(meta_div[,c("q1_j", "q1_v")])

In [19]:
save(meta_div, file = paste0(datapath,"capTCRseq/divstats_immunarch.RData" ))

## Create metadata for immunarch

In [20]:
load(file = paste0(datapath,"capTCRseq/divstats_immunarch.RData" ))

In [21]:
metatmp <- merge(int_load$meta, meta_div, by.x = "Sample", by.y = "basename")

In [ ]:
meta_div$

In [22]:
table(meta_div$cancergroup, useNA = "always")


Leukemia      LFS Lymphoma   Normal    Solid     <NA> 
     394       10      148       14      275        0 

In [23]:
table(metatmp$cancergroup, useNA = "always")


Leukemia      LFS Lymphoma   Normal    Solid     <NA> 
     275       10      103       14      183        0 

In [24]:
meta_div[ !meta_div$sample_id %in% metatmp$sample_id,]

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,Pass,CD3,⋯,q0_aa,q0_v,q0_j,q0_aav,q0_vj,q1_aa,q1_v,q1_j,q1_aav,q1_vj
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,CHP_320_01,CHP_320,X01,8.9,0.00,6.0,AML,8.9,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15,CHP_320_02,CHP_320,X02,3.5,0.00,6.0,AML,3.5,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
16,CHP_320_03,CHP_320,X03,13.2,0.00,6.0,AML,13.2,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
17,CHP_320_04,CHP_320,X04,NA,0.00,6.0,AML,x,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
18,CHP_320_05,CHP_320,X05,0.0,0.00,6.0,AML,0,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
19,CHP_321_01,CHP_321,X01,0.0,0.00,16.0,HD,0,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
20,CHP_321_02,CHP_321,X02,0.0,0.00,16.0,HD,0,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
21,CHP_321_03,CHP_321,X03,0.0,0.00,16.0,HD,0,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22,CHP_321_04,CHP_321,X04,0.0,0.00,16.0,HD,0,NA,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [25]:
#256 samples have flow data but either were not sequenced or didnt recover any productive cdr3

In [26]:
int_load$meta <- as_tibble(metatmp)

In [27]:
dim(int_load$meta)

length(int_load$data)

[1] 585  68

[1] 589

In [28]:
names(int_load$data)[! names(int_load$data) %in% int_load$meta$Sample]

[1] "CHP_303-05-PBMC-DNA" "CHP_315-05_cfDNA"    "CHP_315-05-PBMC-DNA"
[4] "CHP_319-05-PBMC-DNA"

In [29]:
#remove those old samples (patient numbers < 320)
rmsmpl <- c('CHP_303-05-PBMC-DNA', 'CHP_315-05_cfDNA', 'CHP_315-05-PBMC-DNA', 'CHP_319-05-PBMC-DNA')

In [30]:
int_load$data <- int_load$data[ !names(int_load$data) %in% rmsmpl]

In [31]:
length(int_load$data)

[1] 585

In [34]:
int_load$data <- int_load$data[ int_load$meta$Sample]

In [35]:
save(int_load, file = paste0(datapath, "capTCRseq/int_load_meta.RData"))